In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import pandas as pd
import numpy as np
from AutoLLM._utils.general import load_df

# Import Data

In [5]:


df = load_df("./_data/Ecommerce/ecommerceDataset.csv")
df

,Label,Text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...
...,...,...
50420,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...
50421,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...
50422,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...
50423,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou..."


## EDA

In [6]:
all_labels = list(set(df['Label']))
for l in all_labels:
    print(f"{l}: {len(df[df['Label'] == l]) / len(df) * 100:.2f}%")

Clothing & Accessories: 17.20%
Electronics: 21.06%
Books: 23.44%
Household: 38.30%


## Draw examples from data

In [7]:
def get_example(df, num=1, label=None):
    df = df.copy()
    all_labels =  df['Label'].unique()
    if label is not None:
        df = df[df['Label'] == label]
    return df.sample(num)


df_sample = get_example(df, num=5, label=None)
df_sample

,Label,Text
46551,Electronics,Procus Rush 2.0 16MP 4K HD Action Camera Water...
5285,Household,"Saral Home Soft Microfiber Anti Skid Bathmats,..."
37614,Clothing & Accessories,Dada Shopy Comfort Fit Rayon Cotton Pant Palaz...
34803,Clothing & Accessories,PAffy Steel and Fabric Multi-Purpose Shoe Rack...
48557,Electronics,AmazonBasics 3.5mm to 2-Male RCA Adapter cable...


# Load model

In [8]:
from AutoLLM.Models.llama3_model import Llama3InstructModel

l3m = Llama3InstructModel('./model_store/')
l3m.load_model_from_path("Llama3.2_3B_Instruct_GPTQ_4bit_Vortex-v3")

Available models: ['Llama3.1_8B_Instruct_GPTQ_INT4', 'Llama3.2_3B_Instruct_GPTQ_4bit_Vortex-v3']
Flash Attention is not available.


c:\Users\alexr\miniconda3\envs\llm2\Lib\site-packages\transformers\quantizers\auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)
CUDA extension not installed.
CUDA extension not installed.
c:\Users\alexr\miniconda3\envs\llm2\Lib\site-packages\transformers\modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


(LlamaForCausalLM(
   (model): LlamaModel(
     (embed_tokens): Embedding(128256, 3072)
     (layers): ModuleList(
       (0-27): 28 x LlamaDecoderLayer(
         (self_attn): LlamaSdpaAttention(
           (rotary_emb): LlamaRotaryEmbedding()
           (k_proj): QuantLinear()
           (o_proj): QuantLinear()
           (q_proj): QuantLinear()
           (v_proj): QuantLinear()
         )
         (mlp): LlamaMLP(
           (act_fn): SiLU()
           (down_proj): QuantLinear()
           (gate_proj): QuantLinear()
           (up_proj): QuantLinear()
         )
         (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
         (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
       )
     )
     (norm): LlamaRMSNorm((3072,), eps=1e-05)
     (rotary_emb): LlamaRotaryEmbedding()
   )
   (lm_head): Linear(in_features=3072, out_features=128256, bias=False)
 ),
 PreTrainedTokenizerFast(name_or_path='./model_store/Llama3.2_3B_Instruct_GPTQ_4bit_Vortex-v3', vocab_size=12800

# Simple Classification

## Train-Test-Split

In [9]:
from sklearn.model_selection import train_test_split


def split_dataframe(df, test_size=0.2, random_state=None, stratify_col=None):
    """
    Splits a DataFrame into training and testing sets.

    Parameters:
    df (pd.DataFrame): The input DataFrame to be split.
    test_size (float): Proportion of the dataset to include in the test split (default is 0.2).
    random_state (int): Seed used by the random number generator (default is None).
    stratify_col (str): Column to stratify by for stratified sampling (default is None).

    Returns:
    pd.DataFrame, pd.DataFrame: Training and testing DataFrames.
    """
    stratify = df[stratify_col] if stratify_col else None
    train_df, test_df = train_test_split(
        df,
        test_size=test_size,
        random_state=random_state,
        stratify=stratify
    )
    return train_df, test_df

df_train, df_test = split_dataframe(df, stratify_col="Label")
display(df_train)
display(df_test)

,Label,Text
42747,Electronics,Toshiba Canvio Connect II 2TB USB 3.0 External...
6824,Household,Kuber Industries PVC 6 Seater Transparent Dini...
44553,Electronics,boAt Stone 200 Portable Bluetooth Speakers (Bl...
41243,Electronics,"lenovo 65W Slim Port AC Adapter (in) ""Boost yo..."
46030,Electronics,Docooler Fotga 82mm ND Filter Slim Fader Varia...
...,...,...
26401,Books,Human Resource Management
43886,Electronics,AmazonBasics 8-Feet Digital Audio Coaxial Cab...
29637,Books,Oswaal Karnataka PUE Solved Papers II PUC Bio...
29255,Books,The Official Guide to the GRE General Test


,Label,Text
8721,Household,EarthenMetal Handcrafted Bud Shaped Glittering...
26731,Books,The Quick and Easy Way to Effective Speaking
36378,Clothing & Accessories,Perfectblue Women's Linen Saree With Blouse Pi...
3295,Household,VARSHINE Excel Happy Home Automatic Cut Off/on...
14098,Household,BLACK+DECKER Advanced Handheld Garment / Fabri...
...,...,...
32992,Clothing & Accessories,fourgee Women's Skinny Jeans FOURGEE is a bran...
4836,Household,Friends Office Automation Enchanting Moments G...
8727,Household,Generic Lytzon 3-Watt Outdoor Wall LED Light (...
50370,Electronics,UNMCORE IPhone 8 Pin Lightning To USB Fast Dat...


## Naive Classification

In [14]:


config = {
    "sample_subprompt": [
        {
            "name": "Description",
            "description": "Description of item listing on e-commerce site.",
        },
        # {
        #     "name": "Reasoning",
        #     "description": "Reasoning for the answer",
        # },
        {
            "name": "Label",
            "description": "Answer as one of the following prompts only: {Clothing & Accessories, Electronics, Books, Household}",
        }
    ],
    "chat": True,
    "separator": "\n\n=====\n\n",
}

config["sample_subprompt"]

[{'name': 'Description',
  'description': 'Description of item listing on e-commerce site.'},
 {'name': 'Label',
  'description': 'Answer as one of the following prompts only: {Clothing & Accessories, Electronics, Books, Household}'}]

In [16]:
from AutoLLM.prompt_generators.classifier_prompt_generator import ClassifierPromptGenerator
cpg = ClassifierPromptGenerator()
cpg.load_config(config)
cpg.load_tokenizer(l3m.tokenizer)
cpg.load_library()

cpg.build_component_system_message('You are a helpful AI assistant.')
cpg.build_component_instruction(
    instruction='Classify the sentiment of the following text as positive or negative.',
    chain_of_thought='Provide your reasoning for the answer.',
    echo='Repeat the question before you answer.',
)
cpg.build_component_format()
# cpg.build_component_few_shot_examples(few_shot_examples)
cpg.build_component_input_text(input)
cpg.build_component_guide()
print(cpg.build_prompt())

ValueError: Tokenizer must be set if chat is True. Load tokenizer first.

## Build few shot

In [ ]:


config = {
    "sample_subprompt": [
        {
            "name": "Description",
            "description": "Description of item listing on e-commerce site.",
        },
        # {
        #     "name": "Reasoning",
        #     "description": "Reasoning for the answer",
        # },
        {
            "name": "Label",
            "description": "Answer as positive or negative only",
        }
    ],
    "chat": False,
    "separator": "\n\n=====\n\n",
}
few_shot_examples = [
    {
        "Context": "I am happy",
        "Reasoning": "The label is positive because of the word happy.",
        "Answer": "positive"
    },
    {
        "Context": "I am angry",
        "Reasoning": "The label is negative because of the word angry.",
        "Answer": "negative"
    },
    {
        "Context": "I am sad",
        "Reasoning": "The label is negative because of the word sad.",
        "Answer": "negative"
    },
]

input = {
    "Context": "I am satisfied with the product",
}

cpg = ClassifierPromptGenerator()
cpg.load_config(config)
cpg.load_library()

cpg.build_component_system_message('I am a classifier that classifies the sentiment of a given text as positive or negative.')
cpg.build_component_instruction(
    instruction='Classify the sentiment of the following text as positive or negative.',
    chain_of_thought='Provide your reasoning for the answer.',
    echo='Repeat the question before you answer.',
)
cpg.build_component_format()
# cpg.build_component_few_shot_examples(few_shot_examples)
cpg.build_component_input_text(input)
cpg.build_component_guide()
print(cpg.build_prompt())

# cpg.sample_subprompt.build_multiple_examples(few_shot_examples)

I am a classifier that classifies the sentiment of a given text as positive or negative.

=====

Classify the sentiment of the following text as positive or negative. Provide your reasoning for the answer. Repeat the question before you answer.

=====

Format:

Description: Description of item listing on e-commerce site.

Label: Answer as positive or negative only

=====

Description: 


#

# End